<a href="https://colab.research.google.com/github/eumoas/Data-Engineer/blob/main/spark_cloud_storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
!pip install gcsfs

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F

import pandas as pd
from google.cloud import storage
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
serviceAccount = '/content/etl-bc12-97ca935696d1.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
client = storage.Client()

#CRIAR UMA VARIÁVEL CHAMADA BUCKET QUE VAI RECEBER O NOME DA BUCKET DO CLOUD STORAGE
bucket = client.get_bucket('datasets_pyspark')
#USAR O MÉTODO BLOB PARA RETORNAR O NOME DO ARQUIVO (JSON, CSV, PARQUET)
bucket.blob('arquivo_geral.csv')
#CRIA UMA VARIÁVEL PATH PARA COLOCAR O CAMINHO DO CSV
path = 'gs://datasets_pyspark/arquivo_geral.csv'

df_pandas = pd.read_csv(path, sep=';')

In [ ]:
df_pandas.head()

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
0,Norte,RO,2020-01-30,0,0,0,0
1,Norte,RO,2020-01-31,0,0,0,0
2,Norte,RO,2020-02-01,0,0,0,0
3,Norte,RO,2020-02-02,0,0,0,0
4,Norte,RO,2020-02-03,0,0,0,0


In [ ]:
spark = (
    SparkSession.builder
                .master('local')
                .appName('cloudStorage')
                .config('spark.ui.port', '4050')
                .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.22.0')
                .getOrCreate()
)

In [ ]:
df = ( spark.read
      .format('csv')
      .option('header', 'true')
      .option('inferschema', 'true')
      .option('delimiter', ';')
      .load('https://storage.googleapis.com/datasets_pyspark/arquivo_geral.csv')
)

Py4JJavaError: ignored

In [ ]:
df_spark = spark.createDataFrame(df_pandas)

In [ ]:
df_spark.show()

+------+------+----------+----------+---------------+-----------+----------------+
|regiao|estado|      data|casosNovos|casosAcumulados|obitosNovos|obitosAcumulados|
+------+------+----------+----------+---------------+-----------+----------------+
| Norte|    RO|2020-01-30|         0|              0|          0|               0|
| Norte|    RO|2020-01-31|         0|              0|          0|               0|
| Norte|    RO|2020-02-01|         0|              0|          0|               0|
| Norte|    RO|2020-02-02|         0|              0|          0|               0|
| Norte|    RO|2020-02-03|         0|              0|          0|               0|
| Norte|    RO|2020-02-04|         0|              0|          0|               0|
| Norte|    RO|2020-02-05|         0|              0|          0|               0|
| Norte|    RO|2020-02-06|         0|              0|          0|               0|
| Norte|    RO|2020-02-07|         0|              0|          0|               0|
| No